1. Implement decision tree regressor from skratch
2. Apply tree on dataset
3. Compare results with DecisionTreeRegressor from sklearn

## My decision tree
Strategy of missing values imputation: go both ways
Criterions: mse, mae
Regularization: main pre-pruning hyperparameters (min_samples_leaf, max_depth)

In [1]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None, impurity=None, n_samples = None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right    
        self.impurity = impurity
        self.n_samples =  n_samples

        # if this is leaf
        self.value = value

class DecisionTreeRegressor(BaseEstimator, RegressorMixin):    
    def __init__(self, min_samples_leaf=5, max_depth=5, criterion='mse'):
        super().__init__()
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.criterion = criterion
        self.root = None

    def fit(self, X, y):        
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, current_depth=0):        
        n_samples = X.shape[0]        

        if n_samples < self.min_samples_leaf or current_depth >= self.max_depth:
            value = self._create_leaf_value(y)
            return Node(value=value)
                
        best_feature, best_threshold, best_impurity = self._find_best_split(X, y)

        if np.isnan(best_feature) or np.isnan(best_threshold):
            value = self._create_leaf_value(y)
            return Node(value=value)
        

        X_l_cond = X[:, best_feature] <= best_threshold 
        X_r_cond = X[:, best_feature] > best_threshold
        nan_mask = np.isnan(X[:, best_feature])


        X_l = X[X_l_cond | nan_mask]
        X_r = X[X_r_cond | nan_mask]
        y_l = y[X_l_cond | nan_mask]
        y_r = y[X_r_cond | nan_mask]

        return Node(
            feature_index=best_feature, 
            threshold=best_threshold, 
            impurity=best_impurity, 
            left=self._build_tree(X_l, y_l, current_depth=current_depth + 1),
            right=self._build_tree(X_r, y_r, current_depth=current_depth + 1),
            n_samples=n_samples,
            value=None,            
            )
    def _find_best_split(self, X, y):        
        m_impurity = X.shape[0] * self._calculate_impurity(y)
        best_impurity = -float('inf')
        best_feature, best_threshold = np.nan, np.nan

        for j in range(X.shape[1]):
            for t in X[:, j]:
                if np.isnan(t):
                    continue                
                X_l_cond = X[:, j] <= t                
                X_r_cond = X[:, j] > t                
                nan_mask = np.isnan(X[:, j])
                y_l = y[X_l_cond | nan_mask]
                y_r = y[X_r_cond | nan_mask] 
                if y_l.shape[0] == 0 or y_r.shape[0] == 0:
                    continue
                l_impurity = y_l.shape[0] * self._calculate_impurity(y_l) 
                r_impurity = y_r.shape[0] * self._calculate_impurity(y_r)
                
                impurity = m_impurity - l_impurity - r_impurity
                if impurity > best_impurity:
                    best_impurity = impurity
                    best_feature = j
                    best_threshold = t

        return (best_feature, best_threshold, best_impurity)

    def _calculate_impurity(self, y):        
        if self.criterion == 'mse':
            return np.mean((y - np.mean(y)) ** 2)            
        elif self.criterion == 'mae':
            return np.mean(np.abs(y - np.median(y)))            
        else:
            raise ValueError("Criterion must be 'mse' or 'mae'")

    def _create_leaf_value(self, y):        
        if self.criterion == 'mse':
            return np.mean(y)
        elif self.criterion == 'mae':
            return np.median(y)     
        else:
            raise ValueError("Criterion must be 'mse' or 'mae'")

    def predict(self, X):                        
        return np.array([self._traverse_tree(x, self.root) for x in X])        

    def _traverse_tree(self, x, node):    
        node = self.root         
        while node.value is None:
            j = node.feature_index
            t = node.threshold
            if np.isnan(x[j]):
                left_value = self._traverse_tree(x, node.left)                
                right_value = self._traverse_tree(x, node.right)
                return (left_value * node.left.n_samples + right_value * node.right.n_samples) / node.n_samples

            if x[j] <= t:
                # go to left
                node = node.left
            else:
                # go left
                node = node.right
            
        return node.value

In [2]:
from sklearn.model_selection import train_test_split

X = np.random.rand(20, 2,) * 10
# X[[10, 13, 15], 0] = np.nan
# X[1, 1] = np.nan
y = 2 * X + 1 + np.random.randn(20, 1) * 2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
my_tree = DecisionTreeRegressor()
my_tree.fit(X, y)

print(my_tree.predict(X_test))

[ 9.1878073   9.1878073  11.16291517  9.1878073 ]


### Let's test the model on a toy dataset (taken from Hyperparameter Tuning / layer_1.ipynb).
I previously analyzed the data there and concluded that MAE is a better metric for this dataset, since the target distribution is not normal.

In [3]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=42)

In [4]:
my_tree.fit(X_train, y_train)

In [6]:
from sklearn.metrics import mean_absolute_error, r2_score
print('r2 on train', r2_score(y_train, my_tree.predict(X_train)))
print('r2 on test', r2_score(y_test, my_tree.predict(X_test)))
print('mae', mean_absolute_error(y_test, my_tree.predict(X_test)))

r2 on train 0.6334140840815334
r2 on test 0.5072851562053947
mae 41.323487205180186


let's tune hyperparameters!

In [7]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'min_samples_leaf': [2,  5, 8],
    'max_depth': [5, 20, 50, 80],
    'criterion': ['mse', 'mae'],
}
cv = GridSearchCV(estimator=my_tree, param_grid=param_grid, cv=5, verbose=2, scoring='neg_mean_absolute_error')
cv.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=2; total time=   1.0s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=2; total time=   1.0s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=2; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=2; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=2; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=5; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=5; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=5; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=5; total time=   0.9s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=5; total time=   0.6s
[CV] END .....criterion=mse, max_depth=5, min_samples_leaf=8; total time=   0.5s
[CV] END .....criterion=mse, max_depth=5, min_s

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [5, 20, 50, 80],
                         'min_samples_leaf': [2, 5, 8]},
             scoring='neg_mean_absolute_error', verbose=2)

In [8]:
print('best params', cv.best_params_)
print('best mae', -cv.best_score_)
print('mae on test', mean_absolute_error(y_test, cv.best_estimator_.predict(X_test)))
print('r2 on train', r2_score(y_train, cv.best_estimator_.predict(X_train)))
print('r2 on test', r2_score(y_test, cv.best_estimator_.predict(X_test)))

best params {'criterion': 'mse', 'max_depth': 5, 'min_samples_leaf': 8}
best mae 54.59294094398183
mae on test 41.323487205180186
r2 on train 0.6306227701161955
r2 on test 0.5072851562053947


In [9]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(criterion='absolute_error')
tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='absolute_error')

In [10]:
print(mean_absolute_error(y_test, tree.predict(X_test)))
print('r2 on test', r2_score(y_test, tree.predict(X_test)))
print('r2 on train', r2_score(y_train, tree.predict(X_train)))

73.55555555555556
r2 on test -0.4405446491354832
r2 on train 1.0


The model is overfitted — the R2 score indicates that.

In [12]:
param_grid = {
    'min_samples_leaf': [2,  5, 8],
    'max_depth': [5, 20, 50, 80],  
    'criterion': ['squared_error', 'absolute_error'],  
}
cv = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5, verbose=2, scoring='neg_mean_absolute_error')
cv.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=5, min_samples_leaf=8; total time=

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(criterion='absolute_error'),
             param_grid={'criterion': ['squared_error', 'absolute_error'],
                         'max_depth': [5, 20, 50, 80],
                         'min_samples_leaf': [2, 5, 8]},
             scoring='neg_mean_absolute_error', verbose=2)

In [13]:
print('best params', cv.best_params_)
print('best mae', -cv.best_score_)
print('mae on test', mean_absolute_error(y_test, cv.best_estimator_.predict(X_test)))
print('r2 on test', r2_score(y_test, cv.best_estimator_.predict(X_test)))

best params {'criterion': 'squared_error', 'max_depth': 5, 'min_samples_leaf': 5}
best mae 52.055494125813574
mae on test 46.50561563842266
r2 on test 0.4331281148119487


## Good very good!